# Imports

In [5]:
from music_downloader import MusicDownloader
from utils import Credentials

from joblib import Parallel, delayed
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import pandas as pd
import os

# Global variables

In [6]:
DRIVER_PATH = './chromedriver'
SCOPE = "user-read-recently-played"
SPOTIFY_DOWNLOADER_WEBSITE =  'https://www.soundloaders.com/spotify-downloader/'
OUTPUT_DIR_NAME = 'mouka'


playlist_urls = [
    'https://open.spotify.com/playlist/5PQ03g9b9jFvrzE7OLAHPH?si=c6db43bad93d4308',
    'https://open.spotify.com/playlist/7CtI3MSMDCoHtCwpQ5rWCO?si=29a6a148f2d24496',
    'https://open.spotify.com/playlist/1RanMy39MDxrA1inDDP59G?si=a694e5e277e940da',
    'https://open.spotify.com/playlist/6gRwrLeRS9FKRjQWYtPf37?si=d42308d8144f41a6'
]

# Main

In [8]:
cred = Credentials()

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36")
WINDOW_SIZE = "920,800"
# options.add_argument("--headless")
options.add_argument("--windo>w-size=%s" % WINDOW_SIZE)
options.add_argument("--mute-audio")

dir = os.path.join(os.getcwd(), OUTPUT_DIR_NAME)
prefs = {
            "download.default_directory" : OUTPUT_DIR_NAME,
            "savefile.default_directory": OUTPUT_DIR_NAME,}

options.add_experimental_option("prefs", prefs)
options.add_experimental_option('excludeSwitches', ['enable-logging'])
downloader = MusicDownloader(
        output_dir_name=OUTPUT_DIR_NAME, 
        cred=cred, 
        scope=SCOPE,
        spotify_downloader_website=SPOTIFY_DOWNLOADER_WEBSITE,
        driver_path=DRIVER_PATH
        )

downloader.options = options
downloader.dir = dir

In [4]:
downloader.dir

'C:\\Users\\emili\\dev\\songs-loader\\mouka'

In [9]:
artists = []
names = []
urls = []
    
for playlist_url in playlist_urls:
    
    results = downloader.sp.playlist(playlist_url)
    playlist = results['tracks']['items']

    
    for song in playlist:
        track = song['track']
        artist = track['artists'][0]['name']
        name = track['name']
        url = track['external_urls']['spotify']

        artists.append(artist)
        names.append(name)
        urls.append(url)
        print(artist, " – ", name, " – ", url)

Joseph Ray  –  Changing Lanes - Edit  –  https://open.spotify.com/track/5i2lSzyNHNCeqpqtg6oiIQ
andhim  –  Nice to Meet You  –  https://open.spotify.com/track/1AuaqlZieND0Ydk2xTZRqe
Roy Rosenfeld  –  Kala  –  https://open.spotify.com/track/3g1jQEYJMHrJO9MWc6EBj9
Nico de Andrea  –  Wish We Could  –  https://open.spotify.com/track/39zxQPgYKDzxMHtngTDiNK
ASHER SWISSA  –  I Don't Want - Rafael Cerato Remix  –  https://open.spotify.com/track/0HFkgUOdXhlnmT36nSSMDt
ASHER SWISSA  –  I Don't Want - Alexey Union Remix  –  https://open.spotify.com/track/2EZhumeTMfvSXwNSLoEd6J
Black Brejcha  –  2020 Vision  –  https://open.spotify.com/track/1kk0h0yd5WPU06oqjTwMW1
ASHER SWISSA  –  I Don't Want  –  https://open.spotify.com/track/2HsivEVgip05lzSXp2hKfB
Spada  –  Latex  –  https://open.spotify.com/track/4SQikbfT8kPny8aencZqjW
&friends  –  Ode Ireti - Nitefreak Remix  –  https://open.spotify.com/track/7IKCYYmFabJPaFPXn4atBX
Tube & Berger  –  Alive  –  https://open.spotify.com/track/26To29L2K4hJtH4wMg5e

In [10]:
summary = pd.DataFrame(
            {
                "name": names,
                "artist": artists,
                "url": urls,
                })

summary = summary.drop_duplicates()

In [6]:
summary

,name,artist,url
0,Changing Lanes - Edit,Joseph Ray,https://open.spotify.com/track/5i2lSzyNHNCeqpq...
1,Nice to Meet You,andhim,https://open.spotify.com/track/1AuaqlZieND0Ydk...
2,Kala,Roy Rosenfeld,https://open.spotify.com/track/3g1jQEYJMHrJO9M...
3,Wish We Could,Nico de Andrea,https://open.spotify.com/track/39zxQPgYKDzxMHt...
4,I Don't Want - Rafael Cerato Remix,ASHER SWISSA,https://open.spotify.com/track/0HFkgUOdXhlnmT3...
...,...,...,...
115,Eternity,Anyma,https://open.spotify.com/track/4XzeThE3txvCBIr...
116,Would You,Y.V.E. 48,https://open.spotify.com/track/0O1AS0YY1abFpXY...
117,Laroc,Scorz,https://open.spotify.com/track/022a1XbhELRTggf...
118,Light Of Day (feat. Ólafur Arnalds) - Parra fo...,ODESZA,https://open.spotify.com/track/6f8gNNCNSveUl4g...


In [12]:
summ = summary.iloc[:10]

In [8]:
print(dir)
os.makedirs(dir)

C:\Users\emili\dev\songs-loader\mouka


In [18]:
Parallel(n_jobs=10)(delayed(downloader.download_song)(url) 
                        for url in summary['url'] )


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [7]:
downloader.download_song(summary['url'].iloc[0])

C:\Users\emili\dev\songs-loader\mouka
Error with https://open.spotify.com/track/5i2lSzyNHNCeqpqtg6oiIQ


AttributeError: 'Options' object has no attribute 'prefs'

In [17]:
import time
driver = downloader._connect_driver()
driver.get(downloader.spotify_downloader_website)
time.sleep(2)
search_bar = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[1]/div[1]/form/input')
search_bar.clear()
search_bar.send_keys(url)
search_bar.send_keys(Keys.ENTER)
time.sleep(2)
download_button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[1]/div[1]/div/div/div[2]/button')
download_button.click()
time.sleep(3)

download_window = '//*[@id="__next"]/main/div/div[1]/div[1]/div[2]/div/div'
try:
    element = WebDriverWait(driver, 5*60).until_not( EC.presence_of_element_located((By.XPATH, download_window)))
    time.sleep(60)
finally:
    driver.quit()

C:\Users\emili\dev\songs-loader\mouka
